In [1]:
# Importing the libraries
import numpy as np
import pandas as pd
import tensorflow as tf

tf.__version__
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler

In [2]:
# Importing the dataset
dataset = pd.read_csv("train_dataset.csv")
X_train = dataset.iloc[:, 0:28].values
y_train = dataset.iloc[:, 28:38].values
y_train = np.roll(y_train, shift=1, axis=1)

print("X: \n", X_train)
print("y: ", y_train)

X: 
 [[0 1 1 0 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 0 1 1 0]
 [0 0 1 0 0 1 1 0 1 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0]
 [0 1 1 0 1 0 0 1 0 0 0 1 0 0 1 0 0 1 0 0 1 0 0 0 1 1 1 1]
 [0 1 1 0 1 0 0 1 0 0 0 1 0 0 1 0 0 0 0 1 1 0 0 1 0 1 1 0]
 [0 0 0 1 0 0 1 0 0 1 0 0 1 0 0 0 1 1 1 1 0 0 1 0 0 0 1 0]
 [1 1 1 1 1 0 0 0 1 0 0 0 1 1 1 0 0 0 0 1 0 0 0 1 1 1 1 0]
 [0 1 1 0 1 0 0 0 1 0 0 0 1 1 1 0 1 0 0 1 1 0 0 1 0 1 1 0]
 [1 1 1 1 0 0 0 1 0 0 0 1 0 0 1 0 0 1 0 0 1 0 0 0 1 0 0 0]
 [0 1 1 0 1 0 0 1 1 0 0 1 0 1 1 0 1 0 0 1 1 0 0 1 0 1 1 0]
 [0 1 1 0 1 0 0 1 1 0 0 1 0 1 1 1 0 0 0 1 0 0 0 1 0 1 1 0]]
y:  [[1 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 1]]


In [3]:
# Scaling the input features (Optional, but helps in faster convergence)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

In [4]:
# Building the Neural Network
model = Sequential()

# Input layer and first hidden layer with 16 neurons and ReLU activation
model.add(Dense(64, input_dim=28, activation="relu"))

# Second hidden layer with 16 neurons and ReLU activation
model.add(Dense(32, activation="relu"))

# Output layer with 10 neurons (for each digit) and softmax activation
model.add(Dense(10, activation="softmax"))

# Compiling the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Training the model
model.fit(X_train, y_train, epochs=110, batch_size=10)

Epoch 1/110
1/1 [==============================] - 0s 404ms/step - loss: 2.4703 - accuracy: 0.0000e+00
Epoch 2/110
1/1 [==============================] - 0s 4ms/step - loss: 2.4018 - accuracy: 0.0000e+00
Epoch 3/110
1/1 [==============================] - 0s 3ms/step - loss: 2.3354 - accuracy: 0.0000e+00
Epoch 4/110
1/1 [==============================] - 0s 4ms/step - loss: 2.2713 - accuracy: 0.0000e+00
Epoch 5/110
1/1 [==============================] - 0s 3ms/step - loss: 2.2091 - accuracy: 0.0000e+00
Epoch 6/110
1/1 [==============================] - 0s 4ms/step - loss: 2.1487 - accuracy: 0.0000e+00
Epoch 7/110
1/1 [==============================] - 0s 3ms/step - loss: 2.0909 - accuracy: 0.0000e+00
Epoch 8/110
1/1 [==============================] - 0s 4ms/step - loss: 2.0369 - accuracy: 0.0000e+00
Epoch 9/110
1/1 [==============================] - 0s 3ms/step - loss: 1.9832 - accuracy: 0.1000
Epoch 10/110
1/1 [==============================] - 0s 3ms/step - loss: 1.9305 - accuracy: 0.

In [11]:
# Loading the testing dataset
test_dataset = pd.read_csv("test_dataset.csv")

# Separating input features and target labels for testing
X_test = test_dataset.iloc[:, 0:28].values
y_test = test_dataset.iloc[:, 28:38].values
y_test = np.roll(y_test, shift=1, axis=1)
# Scaling the test features
X_test = scaler.transform(X_test)

# Evaluating the model performance
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


# Manually entering input for prediction
manual_input = np.array(
    [
        [
            0,
            1,
            1,
            0,
            1,
            0,
            0,
            1,
            1,
            0,
            0,
            1,
            0,
            1,
            1,
            1,
            0,
            0,
            0,
            1,
            0,
            0,
            0,
            1,
            0,
            1,
            1,
            0,
        ]
    ]
)
manual_input_scaled = scaler.transform(manual_input)

# Predicting the class for the manual input
prediction = model.predict(manual_input_scaled)
print(f"Prediction for the manual input: {prediction}")
predicted_class = np.argmax(prediction, axis=1)
print(f"Predicted class for the manual input: {predicted_class[0]}")

# Saving the model for future use
model.save("digit_recognition_model.h5")

1/1 [==============================] - 0s 26ms/step - loss: 0.0423 - accuracy: 1.0000
Test Accuracy: 100.00%
1/1 [==============================] - 0s 19ms/step
Prediction for the manual input: [[3.0814644e-03 7.8604771e-03 1.8138834e-03 2.3662610e-02 8.5758994e-04
  2.4582059e-03 3.7898449e-04 1.1887305e-03 3.2295284e-03 9.5546848e-01]]
Predicted class for the manual input: 9


In [6]:
# import tkinter as tk

# # Initialize the main window
# root = tk.Tk()
# root.title("4x7 Grid - Click to Toggle Cells")

# # Constants
# ROWS = 7
# COLS = 4

# # Store the cell states (0 for white, 1 for black)
# grid_data = [[0 for _ in range(COLS)] for _ in range(ROWS)]


# # Function to toggle cell color and update grid data
# def toggle_cell(row, col):
#     # Toggle the cell state between 0 and 1
#     grid_data[row][col] = 1 - grid_data[row][col]

#     # Update the cell color
#     color = "black" if grid_data[row][col] == 1 else "white"
#     cells[row][col].configure(bg=color)


# # Function to convert the grid to a vector and display it
# def convert_to_vector():
#     vector = [grid_data[row][col] for row in range(ROWS) for col in range(COLS)]
#     vector_str = ",".join(map(str, vector))

#     # Clear the Entry widget and insert the new vector string
#     vector_entry.delete(0, tk.END)
#     vector_entry.insert(0, vector_str)


# # Create cells (buttons) and add them to the grid
# cells = []
# for row in range(ROWS):
#     row_cells = []
#     for col in range(COLS):
#         cell = tk.Button(
#             root,
#             width=4,
#             height=2,
#             bg="white",
#             command=lambda r=row, c=col: toggle_cell(r, c),
#         )
#         cell.grid(row=row, column=col)
#         row_cells.append(cell)
#     cells.append(row_cells)

# # Convert button
# convert_button = tk.Button(root, text="Convert", command=convert_to_vector)
# convert_button.grid(row=ROWS, column=0, columnspan=COLS, pady=10)

# # Entry to display and allow copying of the vector
# vector_entry = tk.Entry(root, width=28, justify="center")
# vector_entry.grid(row=ROWS + 1, column=0, columnspan=COLS, pady=10)

# # Run the application
# root.mainloop()